In [7]:
# 1) Install dependencies
!pip -q install streamlit openai > /dev/null

# 2) Write the Streamlit app
app_code = """
import os
import torch
import streamlit as st
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import openai

# APP CONFIG
st.set_page_config(page_title="AI Resume Bullet Point Generator", page_icon="💼")
st.title("AI Resume Bullet Point Generator")

st.markdown(
\"\"\"
Writing strong, targeted resume content is difficult - especially when tailoring it to a specific job.

Most students and job seekers:
- Reuse generic resume lines
- Do not highlight their key skills effectively

This app combines your inputs (name, job title, and skills) with AI to generate personalized, job-specific bullet points.
The AI understands the context of the role and your skills to produce:
- Industry-appropriate tasks
- Measurable outcomes
- Strong, professional phrasing
\"\"\"
)

# OPENAI API KEY (embedded)

openai.api_key = ""

# PROMPT CREATION

def make_prompt(name, job_title, skills):
    name_line = f"Candidate: {name}\\n" if name else ""
    prompt = (
        f"You are an expert resume writer.\\n"
        f"{name_line}"
        f"Job Title: {job_title}\\n"
        f"Skills: {skills}\\n\\n"
        "Write 5 to 6 strong, job-specific resume bullet points using action verbs, measurable outcomes, and concise professional phrasing.\\n"
        "Each bullet should be one line long, ATS-friendly, and relevant to the job title.\\n"
        "Output only the bullet points."
    )
    return prompt


# OPENAI GENERATION

def generate_bullets_openai(prompt, temperature=0.7):
    from openai import OpenAI
    client = OpenAI(api_key=openai.api_key)


    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are an expert resume writer."},
            {"role": "user", "content": prompt},
        ],
        temperature=temperature,
        max_tokens=400,
    )

    return response.choices[0].message.content.strip()

# USER INPUT FORM

st.subheader("Enter Your Details")

name = st.text_input("Your Name (optional)")
job_title = st.text_input("Job Title You're Applying For")
skills = st.text_area("List Your Key Skills (comma-separated)")

col_a, col_b = st.columns([1,1])
with col_a:
    max_tokens = st.slider("Max tokens (answer length)", 128, 512, 256, step=32)
with col_b:
    temperature = st.slider("Creativity (temperature)", 0.2, 1.0, 0.7, step=0.1)

generate = st.button("Generate Bullet Points")

# GENERATE BULLETS

if generate:
    if not job_title or not skills:
        st.warning("Please enter both job title and skills.")
    else:
        with st.spinner("Generating resume bullet points..."):
            prompt = make_prompt(name, job_title, skills)
            try:
                output = generate_bullets_openai(prompt, temperature)
                st.success("Generated Resume Bullet Points:")
                st.write(output)
                st.download_button("Download Bullets", output, file_name="resume_bullets.txt")
            except Exception as e:
                st.error(f"Error: {e}")


# INFO SECTION

st.divider()
st.caption("Built with Streamlit and OpenAI GPT-3.5-Turbo.")
"""

# Write the app file
with open("app.py", "w") as f:
    f.write(app_code)

# 3) Kill any running Streamlit and start a new one
!pkill -f streamlit || true
!fuser -k 8501/tcp || true
!streamlit run app.py --server.port 8501 --server.headless true &>/dev/null&

# 4) Cloudflare tunnel for public access
import subprocess, time, re

!wget -q -O cloudflared https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared

p = subprocess.Popen(
    ["./cloudflared", "tunnel", "--url", "http://localhost:8501", "--no-autoupdate"],
    stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True
)

url = None
for _ in range(300):
    line = p.stdout.readline()
    if not line:
        time.sleep(0.1)
        continue
    if "trycloudflare.com" in line:
        for token in line.split():
            if token.startswith("https://") and "trycloudflare.com" in token:
                url = token.strip()
                break
    if url:
        break

print("Streamlit app URL:", url if url else "Still starting... check the logs above.")

^C
cloudflared: Text file busy
Streamlit app URL: https://des-cakes-regulations-flying.trycloudflare.com
